# Practice 2 - Fraud - Variables Transformation

After having taken a look at the dataset in the previous notebook (_Practice 2 - Fraud - EDA_). Here we will be transforming and creating new variables in order to have a better and normalize dataset, that would help in the performance of our models in the following notebooks)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import requests
import pandas_profiling
from pandas_profiling.utils.cache import cache_file
from sklearn.impute import KNNImputer
import scipy.stats as ss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer,  ColumnTransformer
# instantiate labelencoder object
le = LabelEncoder()
import category_encoders as ce
import warnings
warnings.filterwarnings('ignore')

We load the raw dataset again. 

In [2]:
df = pd.read_csv('../data/raw/dataset_payments_fraud.csv', sep = ';')
df.head()

,step,type,amount,gender,device,connection_time,nameOrig,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,nameDest,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,man,mac,"0,140039412",C1231006815,black,170136.0,85,160296.36,capital,138,M1979787155,5,1,0.0,0.0,0
1,1,PAYMENT,1864.28,woman,mac,"0,496889534",C1666544295,asian,21249.0,57,19384.72,country,909,M2044282225,1,0,0.0,0.0,0
2,1,TRANSFER,181.00,man,pc,"0,781150327",C1305486145,asian,181.0,66,0.00,capital,2569,C553264065,10,0,0.0,0.0,1
3,1,CASH_OUT,181.00,man,mac,"0,565068378",C840083671,black,181.0,31,0.00,country,1787,C38997010,3,0,21182.0,0.0,1
4,1,PAYMENT,11668.14,unknow,mac,"0,517114493",C2048537720,black,41554.0,90,29885.86,country,3997,M1230701703,8,0,0.0,0.0,0


And as we have done in the previous notebook, we change the connection time decimal separation to "." in order to have it as a float. 

In [3]:
df = df.assign(**{'connection_time': lambda df: df['connection_time'].str.replace(',', '.').astype(float)})

## **Creation of new variables**

#### **Type_Orig_Dest**

On the conclusions of the previous notebook, we stated that at a first glance the variables nameOrig and nameDest did not seem useful, but they were composed by two elements a letter and a series of numbers. For the number series, did not seem to have a relation, but the letters will only vary between two caracters "C" and "M". 

In [4]:
#First we only take the first value of each colum
df['new_col_Orig'] = df['nameOrig'].astype(str).str[0]
df['new_col_Dest'] = df['nameDest'].astype(str).str[0]
#Then we join them in a new variable called 'Type_Orig_Des'
df['Type_Orig_Des'] = df['new_col_Orig'] +df['new_col_Dest']
df['Type_Orig_Des'].head()

0    CM
1    CM
2    CC
3    CC
4    CM
Name: Type_Orig_Des, dtype: object

In [5]:
#Finally we drop the original name columns as well as the auxiliary ones we created.
df.drop(['new_col_Orig', 'new_col_Dest', 'nameDest','nameOrig' ], axis = 1 , inplace = True)
df.head()

,step,type,amount,gender,device,connection_time,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud,Type_Orig_Des
0,1,PAYMENT,9839.64,man,mac,0.140039,black,170136.0,85,160296.36,capital,138,5,1,0.0,0.0,0,CM
1,1,PAYMENT,1864.28,woman,mac,0.496890,asian,21249.0,57,19384.72,country,909,1,0,0.0,0.0,0,CM
2,1,TRANSFER,181.00,man,pc,0.781150,asian,181.0,66,0.00,capital,2569,10,0,0.0,0.0,1,CC
3,1,CASH_OUT,181.00,man,mac,0.565068,black,181.0,31,0.00,country,1787,3,0,21182.0,0.0,1,CC
4,1,PAYMENT,11668.14,unknow,mac,0.517114,black,41554.0,90,29885.86,country,3997,8,0,0.0,0.0,0,CM


For this new variable we will study its behavious with the target variable **isFraud**

In [6]:
df.groupby(['Type_Orig_Des'])['isFraud'].mean()

Type_Orig_Des
CC    0.001644
CM    0.000000
Name: isFraud, dtype: float64

**Mean encoding Type_Orig_Des**

We observe that the only probabily in the dataset to be fraud is when the Origin and Destination are both the type "C". Hence its seems reasonable to do a mean enconding to the variable 

In [7]:
Mean_encoded_type= df.groupby(['Type_Orig_Des'])['isFraud'].mean().to_dict()
  
df['Type_Orig_Des'] =  df['Type_Orig_Des'].map(Mean_encoded_type)

df.head()

,step,type,amount,gender,device,connection_time,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud,Type_Orig_Des
0,1,PAYMENT,9839.64,man,mac,0.140039,black,170136.0,85,160296.36,capital,138,5,1,0.0,0.0,0,0.000000
1,1,PAYMENT,1864.28,woman,mac,0.496890,asian,21249.0,57,19384.72,country,909,1,0,0.0,0.0,0,0.000000
2,1,TRANSFER,181.00,man,pc,0.781150,asian,181.0,66,0.00,capital,2569,10,0,0.0,0.0,1,0.001644
3,1,CASH_OUT,181.00,man,mac,0.565068,black,181.0,31,0.00,country,1787,3,0,21182.0,0.0,1,0.001644
4,1,PAYMENT,11668.14,unknow,mac,0.517114,black,41554.0,90,29885.86,country,3997,8,0,0.0,0.0,0,0.000000


#### **transactions_per_step**

Also during the EDA we realised the importance of the step. Moreover, we have seen that those steps with less transactions have more probability to be fraud, specially during the step 50 to 90. Hence, we find it insightfull create a new variable that counts the number of transactions per step. 

In [8]:
step_fraud = pd.DataFrame(df.groupby(['step'])['isFraud'].mean())

In [9]:
count_step = pd.DataFrame(df['step'].value_counts(), index=step_fraud.index)
count_step.rename(columns={'step':'step','step':'count'}, inplace=True)

In [10]:
insight_step = pd.concat([count_step, step_fraud], axis=1)
insight_step

,count,isFraud
step,,
1,2708,0.005908
2,1014,0.007890
3,552,0.007246
4,565,0.017699
5,665,0.009023
...,...,...
91,8,1.000000
92,10,1.000000
93,4444,0.003600


After confirming what we saw in the steps graph in the EDA, we proceed to include in our dataset the new variable

In [11]:
df= df.assign(**{'transactions_per_step': lambda df: df.groupby('step')['type'].transform('count')})

In [12]:
df.head()

,step,type,amount,gender,device,connection_time,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud,Type_Orig_Des,transactions_per_step
0,1,PAYMENT,9839.64,man,mac,0.140039,black,170136.0,85,160296.36,capital,138,5,1,0.0,0.0,0,0.000000,2708
1,1,PAYMENT,1864.28,woman,mac,0.496890,asian,21249.0,57,19384.72,country,909,1,0,0.0,0.0,0,0.000000,2708
2,1,TRANSFER,181.00,man,pc,0.781150,asian,181.0,66,0.00,capital,2569,10,0,0.0,0.0,1,0.001644,2708
3,1,CASH_OUT,181.00,man,mac,0.565068,black,181.0,31,0.00,country,1787,3,0,21182.0,0.0,1,0.001644,2708
4,1,PAYMENT,11668.14,unknow,mac,0.517114,black,41554.0,90,29885.86,country,3997,8,0,0.0,0.0,0,0.000000,2708


## **Transformation of existing variables**

#### **Gender, Device and Zone**

Here we have three variables, all three categorical and with NA values. After Taking a look at their meaning, we have conclude that NA can be understood as another variable. 

For gender despite having an _unknow_ variable, we understood that as a "prefer to not disclose that information" option, meanwhile the NA as the question has not been asked to the client.

For device, the variable other than the NA has _mac_, _pc_, and _iphone_. The NA can be other types of device as an android phone, or other transactions that do not compile the information such as bizum. 

Finally, for zone the NA values can mean suburban or transactions that use a VPN hence the information cannot be collected. Hence, we understand it as a different value that _country_, _capital_ or _africa_.

Due to the characteristics of these three variables, we understand that a One Hote Encode is the best option

**One Hot Encoding Gender, Device, Zone**

In [13]:
ohe_cols = ['gender', 'device', 'zone']
ohe = OneHotEncoder()
le = LabelEncoder()

df[ohe_cols] = df[ohe_cols].apply(lambda col: le.fit_transform(col))  

transformed_data = ohe.fit_transform(df[ohe_cols])

# the above transformed_data is an array so convert it to dataframe
encoded_data = pd.DataFrame(transformed_data, index=df.index)

# now concatenate the original data and the encoded data using pandas
df = pd.concat([df, encoded_data], axis=1)

In [14]:
#We eliminate the column "0", generated from the ohe
df=df.drop([0], axis=1)
df.head()

,step,type,amount,gender,device,connection_time,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud,Type_Orig_Des,transactions_per_step
0,1,PAYMENT,9839.64,0,1,0.140039,black,170136.0,85,160296.36,1,138,5,1,0.0,0.0,0,0.000000,2708
1,1,PAYMENT,1864.28,2,1,0.496890,asian,21249.0,57,19384.72,2,909,1,0,0.0,0.0,0,0.000000,2708
2,1,TRANSFER,181.00,0,2,0.781150,asian,181.0,66,0.00,1,2569,10,0,0.0,0.0,1,0.001644,2708
3,1,CASH_OUT,181.00,0,1,0.565068,black,181.0,31,0.00,2,1787,3,0,21182.0,0.0,1,0.001644,2708
4,1,PAYMENT,11668.14,1,1,0.517114,black,41554.0,90,29885.86,2,3997,8,0,0.0,0.0,0,0.000000,2708


#### **type**

As we have seen in the EDA, the only two possible types of transactions that are fraud are Cash-Out and Transfer. In order to remark this to the future models, we will do a mean encoding to the variable.

In [15]:
df.groupby(['type'])['isFraud'].mean()

type
CASH_IN     0.000000
CASH_OUT    0.001547
DEBIT       0.000000
PAYMENT     0.000000
TRANSFER    0.006501
Name: isFraud, dtype: float64

**Mean Ecoding Type**

In [16]:
Mean_encoded_type= df.groupby(['type'])['isFraud'].mean().to_dict()
  
df['type'] =  df['type'].map(Mean_encoded_type)
  
df.head()

,step,type,amount,gender,device,connection_time,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud,Type_Orig_Des,transactions_per_step
0,1,0.000000,9839.64,0,1,0.140039,black,170136.0,85,160296.36,1,138,5,1,0.0,0.0,0,0.000000,2708
1,1,0.000000,1864.28,2,1,0.496890,asian,21249.0,57,19384.72,2,909,1,0,0.0,0.0,0,0.000000,2708
2,1,0.006501,181.00,0,2,0.781150,asian,181.0,66,0.00,1,2569,10,0,0.0,0.0,1,0.001644,2708
3,1,0.001547,181.00,0,1,0.565068,black,181.0,31,0.00,2,1787,3,0,21182.0,0.0,1,0.001644,2708
4,1,0.000000,11668.14,1,1,0.517114,black,41554.0,90,29885.86,2,3997,8,0,0.0,0.0,0,0.000000,2708


#### **user_number**

User number is a variable that goes from 1 to 5000. It seems reasonable to convert it into a categorical numerical variable, with ranges that goes from 500 to 500 values ending with 10 different values.

Since we did not encounter any range of user number that was significantly more prone to be fraud, we will perform directly a OneHotEncoder, but this time manually.

**One Hot Encoding user_number**

In [17]:
df["user_number"]=pd.cut(df.user_number, bins=[0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, float("Inf")],
                        labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ])

#After the transformation it became a str type column, We convert it into integer

df["user_number"] = df["user_number"].astype(str).astype(int)

#### **connection_time**

Connection time, being it a variable that ranges from practically 0 to 1 seconds, and being virtually different for every value in the dataset. Hence in order to be more insgightful, we will range the variable in different slots. These will be generated based on a logarith formula, in order for them to be the same lenghth and based in a mathematical formula.

In [18]:
df['logarithm'] = np.log(df['connection_time']) 
  
df[["connection_time", "logarithm"]].min()

connection_time    2.618800e-08
logarithm         -1.745796e+01
dtype: float64

In [19]:
df[["connection_time", "logarithm"]].max()

connection_time    9.999991e-01
logarithm         -8.670004e-07
dtype: float64

We Observe that the minimum value for the connection time, after the transformation yields a -17.45. Also, the maximum value of connection time yields a value of practically -0. Hence, we transform this yield into a ranging variable, going from "1-2" for the minimun values of the connection time vairble, and to "18-19" for the maximun values.

In [20]:
df["connection"]=pd.cut(df.logarithm, bins=[-18, -17, -16, -15, -14, -13, -12, -11,
                                            -10, -9, -8, -7, -6, -5, -4, -3, -2, -1,
                                            float("Inf")],
                        labels=["1-2","2-3", "3-4", "4-5","5-6", "6-7", "7-8","8-9", "9-10", 
                                "10-11", "11-12","12-13", "13-14", "14-15", "15-16", "16-17", 
                                "17-18", "18-19" ])

In [21]:
df.groupby(['connection'])['isFraud'].mean()

connection
1-2      0.000000
2-3           NaN
3-4           NaN
4-5           NaN
5-6      0.000000
6-7      0.000000
7-8      0.000000
8-9      0.000000
9-10     0.000000
10-11    0.000000
11-12    0.000000
12-13    0.001806
13-14    0.002041
14-15    0.000904
15-16    0.000932
16-17    0.001121
17-18    0.001048
18-19    0.001105
Name: isFraud, dtype: float64

Grouping this ranges respected to the target variable's mean, we find that the 7 higher bins, compound the 100% of the fraud cases. Due to this, we will create a mean encode to the variable.

In [22]:
Mean_encoded_type= df.groupby(['connection'])['isFraud'].mean().to_dict()
  
df['connection'] =  df['connection'].map(Mean_encoded_type)

df.drop(['logarithm'], axis = 1 , inplace = True)

df.head()

,step,type,amount,gender,device,connection_time,race,oldbalanceOrg,age,newbalanceOrig,zone,user_number,user_connections,security_alert,oldbalanceDest,newbalanceDest,isFraud,Type_Orig_Des,transactions_per_step,connection
0,1,0.000000,9839.64,0,1,0.140039,black,170136.0,85,160296.36,1,0,5,1,0.0,0.0,0,0.000000,2708,0.001048
1,1,0.000000,1864.28,2,1,0.496890,asian,21249.0,57,19384.72,2,1,1,0,0.0,0.0,0,0.000000,2708,0.001105
2,1,0.006501,181.00,0,2,0.781150,asian,181.0,66,0.00,1,5,10,0,0.0,0.0,1,0.001644,2708,0.001105
3,1,0.001547,181.00,0,1,0.565068,black,181.0,31,0.00,2,3,3,0,21182.0,0.0,1,0.001644,2708,0.001105
4,1,0.000000,11668.14,1,1,0.517114,black,41554.0,90,29885.86,2,7,8,0,0.0,0.0,0,0.000000,2708,0.001105


## **Variables selection**

As stated before in the EDA, in order to comply with the GDPR, we will drop the race column. 

In [23]:
df.drop(['race'], axis = 1 , inplace = True)

Also, due to the fact that we have already created a variable based on it, we will drop the original connection_time variable. Otherwise, our models will be bias on connection_time as we have two variables that represent the same thing

In [24]:
df.drop(['connection_time'], axis = 1 , inplace = True)

Finally, we will rearrange the variables.

In [25]:
df = df[['step','transactions_per_step', 'type', 'gender', 'device', 'connection','age', 'zone', 'user_number', 
         'user_connections', 'security_alert', 'Type_Orig_Des', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 
         'oldbalanceDest', 'newbalanceDest', 'isFraud']]
df.head()

,step,transactions_per_step,type,gender,device,connection,age,zone,user_number,user_connections,security_alert,Type_Orig_Des,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,2708,0.000000,0,1,0.001048,85,1,0,5,1,0.000000,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,2708,0.000000,2,1,0.001105,57,2,1,1,0,0.000000,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,2708,0.006501,0,2,0.001105,66,1,5,10,0,0.001644,181.00,181.0,0.00,0.0,0.0,1
3,1,2708,0.001547,0,1,0.001105,31,2,3,3,0,0.001644,181.00,181.0,0.00,21182.0,0.0,1
4,1,2708,0.000000,1,1,0.001105,90,2,7,8,0,0.000000,11668.14,41554.0,29885.86,0.0,0.0,0


## **Dataset Normalization**

Having variables that range from 0 to 1 and others which ceiling is in the millions, would be problematic to understand the variable weights for the selected model. Hence, we will normalize those variables that do not range from 0 to 1, to comply with this condition.

In [26]:
variables_to_normalize=['step','transactions_per_step', 'gender', 'device', 'age', 'zone',
                       'user_number', 'user_connections','amount', 'oldbalanceOrg', 
                        'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest' ]

In [27]:
df_to_normalize = df[variables_to_normalize]

# Min-Max Normalization
df_normalized= (df_to_normalize-df_to_normalize.min())/(df_to_normalize.max()-df_to_normalize.min())
 
df_normalized.head()

,step,transactions_per_step,gender,device,age,zone,user_number,user_connections,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,0.0,0.05266,0.000000,0.333333,0.842105,0.333333,0.000000,0.444444,0.000984,0.004369,0.004116,0.000000,0.0
1,0.0,0.05266,0.666667,0.333333,0.547368,0.666667,0.111111,0.000000,0.000186,0.000546,0.000498,0.000000,0.0
2,0.0,0.05266,0.000000,0.666667,0.642105,0.333333,0.555556,1.000000,0.000018,0.000005,0.000000,0.000000,0.0
3,0.0,0.05266,0.000000,0.333333,0.273684,0.666667,0.333333,0.222222,0.000018,0.000005,0.000000,0.000504,0.0
4,0.0,0.05266,0.333333,0.333333,0.894737,0.666667,0.777778,0.777778,0.001167,0.001067,0.000767,0.000000,0.0


We merge the df_to_normalise with the df 

In [28]:
for i in variables_to_normalize:
    df[i] = df_normalized[i]
    
df.head()

,step,transactions_per_step,type,gender,device,connection,age,zone,user_number,user_connections,security_alert,Type_Orig_Des,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,0.0,0.05266,0.000000,0.000000,0.333333,0.001048,0.842105,0.333333,0.000000,0.444444,1,0.000000,0.000984,0.004369,0.004116,0.000000,0.0,0
1,0.0,0.05266,0.000000,0.666667,0.333333,0.001105,0.547368,0.666667,0.111111,0.000000,0,0.000000,0.000186,0.000546,0.000498,0.000000,0.0,0
2,0.0,0.05266,0.006501,0.000000,0.666667,0.001105,0.642105,0.333333,0.555556,1.000000,0,0.001644,0.000018,0.000005,0.000000,0.000000,0.0,1
3,0.0,0.05266,0.001547,0.000000,0.333333,0.001105,0.273684,0.666667,0.333333,0.222222,0,0.001644,0.000018,0.000005,0.000000,0.000504,0.0,1
4,0.0,0.05266,0.000000,0.333333,0.333333,0.001105,0.894737,0.666667,0.777778,0.777778,0,0.000000,0.001167,0.001067,0.000767,0.000000,0.0,0


This is the dataframe that we will use for our models. We export it to csv.

In [29]:
df.to_csv('../data/processed/df_selected.csv')